# Santande Dev Week 2023 (ETL com Python)

## Preparando o ambiente

In [1]:
!python.exe -m pip install --upgrade pip
!pip install pandas
!pip install requests
!pip install json
!pip install openai

^C


'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [2]:
import pandas as pd
import requests
import json
import openai

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/e0/63/b428aaca15fcd98c39b07ca7149e24bc14205ad0f1c80ba2b01835aedde1/pip-23.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 1.3 MB/s eta 0:00:02
   - -------------------------------------- 0.1/2.1 MB 871.5 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/2.1 MB 944.1 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/2.1 MB 1.4 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/2.1 MB 1.4 MB/s eta 0:00:02
   ------ --------------------------------- 0.4/2.1 MB 1.4 MB/s eta 0:00:02
   ---------- ----------------------------- 0.6/2.1 MB 1.7 MB/s eta 0:00:01
   ------------ --------------------------- 0.7/2.1 MB 1.9 MB/s eta 0:00:01
   ---------------------- ----------------- 1.2/2.1 MB 2.9 MB/s eta 0:00:01
   ------------------------- -------------- 1.4/2.1 M

## Extract
Extraindo a lista de IDs de usuário a partir do arquivo CSV. Será feita uma requisição que me retornará todos os usuários

In [3]:
# URL da api
url = 'https://sdw-2023-prd.up.railway.app/users'

In [4]:
# Função que me retorna todos os usuários
def get_users(url):
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

In [6]:
# Recendo a lista de usuários da API transformando em um dataframe para conseguir usar pandas
data = pd.DataFrame(columns=['id','name','account','card','features','news'])
lista_de_usuarios = get_users(url=url)
for user in lista_de_usuarios:
#    print(user)
    for key in data.columns:
        if key not in user:
            user[key] = None
data = pd.concat([data,pd.DataFrame(lista_de_usuarios)],ignore_index=True)

# Transform

In [7]:
# Chave API da OpenAI (Substitua pela sua chave API)
api_key = 'TODO'

In [8]:
# Função para gerar news automaticamente (Baseado no chatGPT)
openai.api_key = api_key
def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')


In [ ]:
# Prrenchendo os valores vazios da coluna news (base de dados já veio com alguns news preenchidos previamente)
for _, row in data.iterrows():
    if row['news'] == []:
        news = generate_ai_news(user=row['name'])
        row['news'] = [{
            "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
            "description": news
        }]

# Load

In [ ]:
# Exportando um Excel com os dados
data.to_excel('data.xlsx',index=False)